Model Used: https://huggingface.co/bert-base-uncased

In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

import sys
sys.path.append('..//..')
from utils.tokenizer import sent_tokenize
from dataset import job_description

# import dataset 
dataset = pd.read_csv('../../dataset/resume_dataset.csv')

# drop unecessary columns
dataset = dataset.drop(columns=['ID', 'Resume_html', 'Category'])

# import job description
job_desc_sentences = job_description.job_description

# import bert
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

d:\Applications\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identic

In [2]:
resume_sentences = sent_tokenize(dataset['Resume_str'][0])

tokenized_resume_sentences = tokenizer(resume_sentences, return_tensors='pt', padding=True, truncation=True)

# resume embeddings
with torch.no_grad():
    outputs = model(**tokenized_resume_sentences)
    resume_embeddings = outputs.last_hidden_state

tokenized_job_desc_sentences = tokenizer(job_desc_sentences, return_tensors='pt', padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**tokenized_resume_sentences)
    job_desc_embeddings = outputs.last_hidden_state

In [3]:
mean_pooled_word_embeddings = [np.mean(sentence, axis=0) for sentence in resume_embeddings]
print(mean_pooled_word_embeddings)
mean_pooled_sentence_embeddings = np.mean(mean_pooled_word_embeddings, axis=0)

print(mean_pooled_sentence_embeddings)

TypeError: mean() received an invalid combination of arguments - got (axis=int, dtype=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)
